In [3]:
from huggingface_hub import login
login()

In [ ]:
from datasets import load_dataset
import json
from transformers import TextStreamer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch


In [8]:
ds = load_dataset("cat-searcher/minif2f-lean4", split="test")
save_path = "./minif2f_saved" 
ds.save_to_disk(save_path)

Saving the dataset (0/1 shards):   0%|          | 0/244 [00:00<?, ? examples/s]

In [13]:
from datasets import load_from_disk
ds = load_from_disk(save_path)

In [ ]:
model_id = "RickyDeSkywalker/TheoremLlama"
tok  = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id)


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [10]:
model_save_path = "./saved_theorem_llama"
tok.save_pretrained(model_save_path)
model.save_pretrained(model_save_path)

/root/assip2025/.venv/lib/python3.10/site-packages/transformers/modeling_utils.py:3737: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [11]:
local_tok = AutoTokenizer.from_pretrained(model_save_path)
local_theoremllama = AutoModelForCausalLM.from_pretrained(model_save_path)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [12]:
streamer = TextStreamer(tok)

In [14]:
import os
PROOF_DIR    = "/assip2025/llama_proofs/LlamaProofs"


In [ ]:
for ex in ds:
    pid = ex["id"]                  
    lean_out = f"{PROOF_DIR}/{pid}.lean"
    if os.path.exists(lean_out):
        print(f"✔︎ {pid} already done, skipping.")
        continue

    header   = ex.get("header", "")
    theorem  = ex["formal_statement"]
    nl_desc  = ex["informal_stmt"]

    prompt = (
        f"-- Informal description\n{nl_desc}\n\n"
        f"-- Formal theorem skeleton\n{theorem}\n\n"
        f"-- Fill in Lean proof after `:= by` or `:=\nby`:\n"
    )

    input_ids = tok(prompt, return_tensors="pt").to(model.device)

    gen_ids = model.generate(
        **input_ids,
        max_new_tokens=512,
        do_sample=False,      # greedy; raise for exploration
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.eos_token_id,
        streamer=streamer,
    )
    proof = tok.decode(gen_ids[0][input_ids["input_ids"].shape[1]:],
                       skip_special_tokens=True)

    # ===== 6. Save .lean file =====
    with open(lean_out, "w") as f:
        f.write(f"{header}\n\n{theorem[:-6]}:= by\n{proof}\n")
    print(f"💾  Saved proof for {pid} → {lean_out}")